<a href="https://colab.research.google.com/github/Noahbisht0/Deep-Learning/blob/main/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from lr_utils import load_dataset
from matplotlib.pyplot import imread
from skimage.transform import resize

train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()
index = 24
#plt.imshow(train_set_x_orig[index])
#print("y = " + str(train_set_y[:,index]) + ",it is a '" + classes[np.squeeze(train_set_y[:,index])].decode('utf-8') +  "' picture")
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]
print("number of training examples: " +str(m_train))
print("number of testing examples: " +str(m_test))
print("height/ width of the images: " +str(num_px))
print("the size of each image: (" + str(num_px) + "," + str(num_px) + ", 3)")
print("train size x: " +str(train_set_x_orig.shape))
print("train size y: " +str(train_set_y.shape))
print("test size x: " +str(test_set_x_orig.shape))
print("test size y: " +str(test_set_y.shape))
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1).T
print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))
train_set_x = train_set_x_flatten/255. 
test_set_x = test_set_x_flatten/255.
def sigmoid(z):
  s = 1 / (1 + np.exp(-z))
  return s
def initialize_with_zeros(dim):
  w = np.zeros((dim,1))
  b = 0
  assert(w.shape ==(dim,1))
  assert(isinstance(b, float) or (isinstance(b,int)))
  return w,b
dim = 2
w,b = initialize_with_zeros(dim)
def propagate(w,b,X,Y):
  m = X.shape[1]
  A = sigmoid(np.dot(w.T,X) + b)
  cost = (-1/m) * np.sum((Y * np.log(A)) + (1-Y) * np.log(1-A), axis = 1)
  dw = (1/m) * np.dot(X,(A-Y).T)
  db = (1/m) * np.sum(A -Y,axis = 1)
  assert(dw.shape == w.shape)
  assert(db.dtype == float)
  cost = np.squeeze(cost)
  assert(cost.shape == ())
  grads = {"dw": dw,
            "db": db}
  return grads,cost
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
grads, cost = propagate(w,b,X,Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))           
def optimize(w,b,X,Y,num_iterations,learning_rate,print_cost = False):
  costs = []
  for i in range(num_iterations):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T,X) + b)
    cost = (-1/m) * np.sum((Y * np.log(A)) + (1-Y) * np.log(1-A), axis = 1)
    dw = (1/m) * np.dot(X,(A-Y).T)
    db = (1/m) * np.sum(A -Y,axis = 1)
    grads = {"dw": dw,"db": db}
    dw = grads['dw']
    db = grads['db']
    w = w - learning_rate * dw
    b = b - learning_rate * db
    if i % 100 == 0:
      costs.append(cost)
    if print_cost and i % 100 == 0:
      print("cost after iterations %i : %f" %(i,cost))
  params = {"w":w,"b":b}
  grads = {"dw":dw,"db":db}
  return params,grads,costs
params, grads,costs = optimize(w,b,X,Y,num_iterations = 100,learning_rate = 0.009,print_cost = False)
print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))  
def predict(w,b,X):
  m = X.shape[1]
  Y_predictions = np.zeros((1,m))
  w = w.reshape(X.shape[0],1)
  A = sigmoid(np.dot(w.T,X)+b)
  for i in range(A.shape[1]):
    if A[0,i] <= 0.5:
      Y_predictions[0,i] = 0
    else:
      Y_predictions[0,i] = 1
    pass
  assert(Y_predictions.shape == (1,m))
  return  Y_predictions
w = np.array([[0.1124579],[0.23106775]])
b = -0.3
X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
print ("predictions = " + str(predict(w, b, X)))
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
  w,b = initialize_with_zeros(X_train.shape[0])
  parameters, grads, costs = optimize(w,b,X_train,Y_train, num_iterations, learning_rate, print_cost) 
  w = parameters["w"]
  b = parameters["b"]
  Y_prediction_test = predict(w,b,X_test)
  Y_prediction_train = predict(w,b,X_train)
  print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
  print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
  d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
  return d
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)  
index = 1
plt.imshow(test_set_x[:,index].reshape((num_px, num_px, 3)))
print("y = "+str(test_set_y[0,index])+ ",1 => cat , 0 => non-cat")
my_image = "b.jpg"
fname = "images/" + my_image
image = np.array(imread(fname))
image = image/255.
my_image = resize(image,(64,64)).reshape((1,num_px*num_px*3)).T
my_predicted_image = predict(d["w"],d["b"],my_image)

plt.imshow(image)
print("y = " + str(np.squeeze(my_predicted_image)) + ", your algorithm predicts a \"" + classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\" picture.")



SyntaxError: ignored